In [29]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

# 方案分享
本方案思路是用2个模型分别训练，之后再融合结果
1. XGB
2. 神经网络

# XGB部分

In [30]:
data=pd.read_csv('../data/data137276/train.csv.zip')
test=pd.read_csv('../data/data137276/test.csv.zip')


In [31]:
data = data.drop(['id', 'timecc'], axis=1)
test = test.drop(['timecc', 'id'],axis=1)

data['kda'] = (data['kills'] + data['assists'] - data['deaths'])/3
data['damage_dif'] = data['totdmgdealt'] - data['totdmgtaken']
data['damagephysic_dif'] = data['physdmgtochamp'] - data['totdmgtaken']
data['damagemagic_dif'] = data['magicdmgtochamp'] - data['physdmgtaken']
data['damagereal_dif'] = data['truedmgtochamp'] - data['truedmgtaken']

test['kda'] = (test['kills'] + test['assists'] - test['deaths'])/3
test['damage_dif'] = test['totdmgdealt'] - test['totdmgtaken']
test['damagephysic_dif'] = test['physdmgtochamp'] - test['totdmgtaken']
test['damagemagic_dif'] = test['magicdmgtochamp'] - test['physdmgtaken']
test['damagereal_dif'] = test['truedmgtochamp'] - test['truedmgtaken']

In [32]:
label=data['win']

In [33]:
train=data
data = train.drop('win', axis=1)

In [34]:
clf =XGBClassifier(
    n_estimators=2000,
    max_depth=11,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='gpu_hist'  # THE MAGICAL PARAMETER
)

In [35]:
eval_set=[(data,label)]
clf.fit(data,label,early_stopping_rounds=1500,eval_metric='error',eval_set=eval_set,verbose=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-error:0.15044
[1]	validation_0-error:0.14374
[2]	validation_0-error:0.13432
[3]	validation_0-error:0.13020
[4]	validation_0-error:0.12655
[5]	validation_0-error:0.12532
[6]	validation_0-error:0.12407
[7]	validation_0-error:0.12289
[8]	validation_0-error:0.12134
[9]	validation_0-error:0.11990
[10]	validation_0-error:0.11926
[11]	validation_0-error:0.11791
[12]	validation_0-error:0.11717
[13]	validation_0-error:0.11603
[14]	validation_0-error:0.11516
[15]	validation_0-error:0.11414
[16]	validation_0-error:0.11334
[17]	validation_0-error:0.11241
[18]	validation_0-error:0.11167
[19]	validation_0-error:0.11021
[20]	validation_0-error:0.10926
[21]	validation_0-error:0.10785
[22]	validation_0-error:0.10593
[23]	validation_0-error:0.10479
[24]	validation_0-error:0.10439
[25]	validation_0-error:0.10321
[26]	validation_0-error:0.10278
[27]	validation_0-error:0.10141
[28]	validation_0-error:0.10062
[29]	validation_0-error:0.10006
[30]	validation_0-error:0.09936
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(label,clf.predict(data))

0.99995

# 神经网络

In [39]:
import paddle

In [40]:
class Classifier(paddle.nn.Layer):
    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Classifier, self).__init__()
        

        self.flatten = paddle.nn.Flatten()
        self.linear_1 = paddle.nn.Linear(34, 68)
        self.linear_2 = paddle.nn.Linear(68, 68)
        self.linear_3 = paddle.nn.Linear(68, 16)
        self.linear_4 = paddle.nn.Linear(16,1)
        self.relu = paddle.nn.ReLU()
#        self.dropout = paddle.nn.Dropout(0.2)

    def forward(self, inputs):
        # print('input', inputs)

        y = self.flatten(inputs)
        y = self.linear_1(y)
        y = self.relu(y)
        y = self.linear_2(y)
        y = self.relu(y)
#        y = self.dropout(y)
        y = self.linear_3(y)
        y = self.relu(y)
        y = self.linear_4(y)

        return y

In [41]:
model = Classifier()
model.train()
opt = paddle.optimizer.Adam(learning_rate=0.1, parameters=model.parameters())
loss_fn = paddle.nn.BCEWithLogitsLoss()

W0908 14:09:16.430028   298 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0908 14:09:16.435662   298 device_context.cc:465] device: 0, cuDNN Version: 7.6.


In [ ]:
EPOCH_NUM = 32   # 设置外层循环次数
BATCH_SIZE = 100  # 设置batch大小
training_data = train_df.iloc[:-5000,].values.astype(np.float32)
val_data = train_df.iloc[-5000:, ].values.astype(np.float32)


if __name__ == '__main__':
    # 初始化一个记录器
    with LogWriter(logdir="./log/scalar_test/train") as writer:
    # 定义外层循环
        for epoch_id in range(EPOCH_NUM):
            # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
            
            np.random.shuffle(training_data)
            
            # 将训练数据进行拆分，每个batch包含10条数据
            mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
            
            # 定义内层循环
            for iter_id, mini_batch in enumerate(mini_batches):
                x = np.array(mini_batch[:, 1:]) # 获得当前批次训练数据
                y = np.array(mini_batch[:, :1]) # 获得当前批次训练标签
                
                # 将numpy数据转为飞桨动态图tensor的格式
                features = paddle.to_tensor(x)
                y = paddle.to_tensor(y)
                
                # 前向计算
                predicts = model(features)
                
                # 计算损失
                loss = loss_fn(predicts, y, )
                avg_loss = paddle.mean(loss)

                acc = (predicts > 0).astype(int).flatten() == y.flatten().astype(int)
                acc = acc.astype(float).mean()

                if iter_id%200==0:

                    print("total_num:{}, epoch: {}, iter: {}, loss is: {}, acc is {}".format(epoch_id*int(BATCH_SIZE)*14+int(iter_id),epoch_id, iter_id, avg_loss.numpy(), acc.numpy()))
                
                writer.add_scalar(tag="acc", step=epoch_id*int(BATCH_SIZE)*14+int(iter_id), value=acc.numpy())
                # 向记录器添加一个tag为`loss`的数据
                writer.add_scalar(tag="loss", step=epoch_id*int(BATCH_SIZE)*14+int(iter_id), value=avg_loss.numpy())            
                # 反向传播，计算每层参数的梯度值
                avg_loss.backward()
                # 更新参数，根据设置好的学习率迭代一步
                opt.step()
                # 清空梯度变量，以备下一轮计算
                opt.clear_grad()

            paddle.save(Classifier,"model"+str(epoch_id)+".pdparams")

In [37]:
pre=clf.predict(test)
pad=pd.DataFrame({'win':pre})
pad.to_csv('submission.csv',index=False,sep=',') \

!zip submission.zip submission.csv

updating: submission.csv (deflated 90%)
